In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize,word_tokenize
import pandas as pd
from docx import Document
import docx2txt
import mammoth
import textract
import spacy
import numpy as np
import PyPDF2
import os
import io

nlp = spacy.load('en_core_web_md')

In [ ]:


def get_email(text):
    # compile helps us to define a pattern for matching it in the text
    r = re.compile(r'[A-Za-z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    return r.findall(str(text))
    
def extract_phone_numbers(text):
    
    text_without_extra_spaces = re.sub(r'\s+', '', text)
    #text_without_extra_spaces_brac=re.sub(r'[()\[\]{}]', '',text_without_extra_spaces)
    print(text_without_extra_spaces)
    phone_regex = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
    phone_numbers = re.findall(phone_regex, text_without_extra_spaces)
    #print(text_without_extra_spaces_brac)
    new_phone_numbers=[]
    for i in phone_numbers:
        i=re.sub(r'[()\[\]{}]', '',i)
        if len(i)>7 and i[0]!='2' and i[0]!='1' and i[0]!='0':
            new_phone_numbers.append(i) 
    return new_phone_numbers

def get_name(txt):
    stops=stopwords.words('english')
    punc=string.punctuation
    punc=[char for char in punc ]
    stops+=punc
    
    skills=pd.read_csv(skills_data_path)
    skills_name=pd.DataFrame(skills['Text'])
    skill_val=skills_name.values
    skills_val = skill_val.reshape(-1).tolist()
    
    split_string =[]
    split_string = txt.split()
    #print(split_string)
    name=" "
    for j in range(len(split_string)):
        
        w=split_string[j]
        #print(j,w,pos_tag([w]))
        if w.lower() not in stops and len(w)!=1 and len(w)!=2 and w.lower() not in skills_val and w.isalpha() and "@" not in w and "gmail.com" not in w :
            #print(w,pos_tag([w]))
            name=w+" "+split_string[j+1]
            break
    return name
    
def clean_institute_name(name):
    # Remove leading/trailing whitespace
    name = name.strip()
    # Replace '\t' and '\n' with a space
    name = re.sub(r'[\t\n]', ' ', name)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name)
    return name
def extract_educational_institutes(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract educational institutes
    educational_institutes = []
    for entity in doc.ents:
        if entity.label_ == "ORG"  and ("university" in entity.text.lower() or "college" in entity.text.lower() or "institute" in entity.text.lower()):  # Check if the entity is an organization
            clean_name = clean_institute_name(entity.text)
            educational_institutes.append(clean_name)

    
    return educational_institutes

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file,strict=False)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text

def extract_text_from_docx(docx_path):
    with open(docx_path, 'rb') as docx_file:
        result = mammoth.extract_raw_text(docx_file)
        text = result.value
    
    return text

def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else: 
        return wordnet.NOUN
        
lemmatizer=WordNetLemmatizer()

def clean_resume(txt):
    stops=stopwords.words('english')
    punc=string.punctuation
    punc=[char for char in punc ]
    stops+=punc
    removed_space=[]
    tok=[]
    clean_resume=[]
    
    for resume in txt : 
        text = re.sub(r'\s+', ' ', resume)
        removed_space.append(text)
    #print(len(removed_space))
    #print(removed_space)

    for resume in removed_space:
        word_tok=[]
        word_tok= word_tokenize(resume)
        tok.append(word_tok)
    #print(len(tok))
    #print(tok)
       
    
    for resume in tok:
        output_words=[]
        for w in resume:
            if w.lower() not in stops and len(w)!=2 and len(w)!=1 :
                pos=pos_tag([w])               # passing the as it is important because if we lower it we might lose some information
                clean_words=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
                output_words.append(clean_words.lower())
        clean_resume.append(output_words)
    
    final_cleaned_res=[]
    for i in clean_resume:
        one_res=" ".join(i)
        final_cleaned_res.append(one_res)
    
    return final_cleaned_res


    


#return fileTXT,file_names







In [ ]:
def Analyse_Resume(Resume_path,skills_data_path,JD_path):
    
    skills=pd.read_csv(skills_data_path)
    skills_name=pd.DataFrame(skills['Text'])
    skill_val=skills_name.values
    skills_val = skill_val.reshape(-1).tolist()
    
    fileTXT=[]
    file_names=[]
    for filename in os.listdir(Resume_path):
        contact_info=[]
        text=""
        if filename.endswith(".pdf") or filename.endswith(".PDF"):
            try:
                text=extract_text_from_pdf(Resume_path+"/"+filename)
                fileTXT.append(text)
                
                name = get_name(text)
                contact_info.append(name)
                phn=extract_phone_numbers(text)
                contact_info.append(phn)
                email=get_email(text)
                contact_info.append(email)
                contact_info.append(Resume_path+"/"+filename)
                file_names.append(contact_info)
                edu=extract_educational_institutes(text)
                contact_info.append(edu)
            
            except Exception:
                print("Error Reading pdf file :" + filename)
        
        elif filename.endswith(".docx"):
            try:
                text=extract_text_from_docx(Resume_path+"/"+filename)
                fileTXT.append(text)
                
                name = get_name(text)
                contact_info.append(name)
                phn=extract_phone_numbers(text)
                contact_info.append(phn)
                email=get_email(text)
                contact_info.append(email)
                contact_info.append(Resume_path+"/"+filename)
                file_names.append(contact_info)
                edu=extract_educational_institutes(text)
                contact_info.append(edu)
                

            except Exception:
                print("Error reading docx file:" + filename)

        elif filename.endswith(".doc"):
            try:
                text=textract.process(Resume_path+"/"+filename).decode('utf-8')
                fileTXT.append(text)

                name = get_name(text)
                contact_info.append(name)
                phn=extract_phone_numbers(text)
                contact_info.append(phn)
                email=get_email(text)
                contact_info.append(email)
                contact_info.append(Resume_path+"/"+filename)
                file_names.append(contact_info)
                edu=extract_educational_institutes(text)
                contact_info.append(edu)
            
            except Exception:
                print('Error reading .doc file :' + filename)


    final_res=clean_resume(fileTXT)
    jd=extract_text_from_pdf(JD_path)
    final_jd= clean_resume([jd])
    jd_by_words=word_tokenize(final_jd[0])
    skills_from_jd=[]
    for i in jd_by_words:
        for j in skills_val:
            if i==j:
                skills_from_jd.append(i)
    two_word_list=[]
    for i in range(len(jd_by_words)-1):
        two_words=""
        two_words=jd_by_words[i]+" "+jd_by_words[i+1]
        two_word_list.append(two_words)
    
    skills_from_jd2=[]
    for i in two_word_list:
        for j in skills_val:
            if i==j:
                skills_from_jd2.append(i)
    final_features_from_jd= skills_from_jd+skills_from_jd2
    new_final_features_from_jd=set(final_features_from_jd)
    new_final_features_from_jd_list=list(new_final_features_from_jd)
    soft_new_final_features_from_jd_list=[]
    tech_new_final_features_from_jd_list=[]
    for j in range(len(new_final_features_from_jd)):
        f=new_final_features_from_jd_list[j]
        if ((skills[skills['Text']==f]['Label']).values[0])==1:
            tech_new_final_features_from_jd_list.append(new_final_features_from_jd_list[j])
        else:
            soft_new_final_features_from_jd_list.append(new_final_features_from_jd_list[j])
    combined_skills=tech_new_final_features_from_jd_list+soft_new_final_features_from_jd_list


    final_res.append(final_jd[0])

    new_final_res_with_prime_words=[]
    new_file_names=[]
    # for java developer
    if 'java' in final_jd[0]:
        for i in range(len(final_res)-1):
            splited_res=[]
            splited_res=final_res[i].split()
            if 'java' in splited_res:
                new_final_res_with_prime_words.append(final_res[i])
                new_file_names.append(file_names[i])
        new_final_res_with_prime_words.append(final_jd[0])
    elif 'legal' in final_jd[0]:
        for i in range(len(final_res)-1):
            splited_res=[]
            splited_res=final_res[i].split()
            if 'legal' in splited_res:
                new_final_res_with_prime_words.append(final_res[i])
                new_file_names.append(file_names[i])
        new_final_res_with_prime_words.append(final_jd[0])
    
    elif 'graphic' in final_jd[0]:
        for i in range(len(final_res)-1):
            splited_res=[]
            splited_res=final_res[i].split()
            if 'graphic' in splited_res:
                new_final_res_with_prime_words.append(final_res[i])
                new_file_names.append(file_names[i])
        new_final_res_with_prime_words.append(final_jd[0])
    
    else:
        new_final_res_with_prime_words=final_res
        new_file_names=file_names

    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(vocabulary = combined_skills,binary=True,ngram_range=(1,2))
    X = vectorizer.fit_transform(new_final_res_with_prime_words)
    X_array = X.toarray()

    # for making feature columns ( that will have skills that are present in the resume)
    soft_skill_feature=[]
    tech_skill_feature=[]
    for i in range(len(X_array)-1):
        soft_skill_feature_for_one_resume=[]
        tech_skill_feature_for_one_resume=[]
        for j in range(len(new_final_features_from_jd)):
            f=new_final_features_from_jd_list[j]
    
            if X_array[i][j]==1:
                if ((skills[skills['Text']==f]['Label']).values[0])==1:
                    tech_skill_feature_for_one_resume.append(new_final_features_from_jd_list[j])
                else:
                    soft_skill_feature_for_one_resume.append(new_final_features_from_jd_list[j])
                    
        soft_skill_feature.append(soft_skill_feature_for_one_resume)
        tech_skill_feature.append(tech_skill_feature_for_one_resume)

    from sklearn.metrics.pairwise import cosine_similarity
    similarity =cosine_similarity(X_array)
    final_similarity=similarity[:,len(new_final_res_with_prime_words)-1]
    for i in range(len(final_similarity)-1):
        new_file_names[i].append(final_similarity[i])
        new_file_names[i].append(soft_skill_feature[i])
        new_file_names[i].append(tech_skill_feature[i])
    final_df=pd.DataFrame(new_file_names,columns=["Name",'Contact Number','Email Address','File Location','Education','Score','Soft Skills','Tech Skill'])
    df_sorted = final_df.sort_values(by=['Score'],ascending=False)
    df_sorted.to_csv('data1.csv', index=False)
    return df_sorted

In [ ]:
Resume_path='/Users/navneetyadav/Downloads/All_resume_new'
skills_data_path='/Users/navneetyadav/Downloads/archive 2/all_data_skill_and_nonskills/all_data_skill_and_nonskills.csv'
JD_path='/Users/navneetyadav/Downloads/JD_Java_Developer.pdf'

df=Analyse_Resume(Resume_path,skills_data_path,JD_path)

In [ ]:
names='/Users/navneetyadav/Downloads/All_resume_new/Legal_Resume- Anjali Sharma..pdf'
if names.endswith(".pdf") or filename.endswith(".PDF"):
    print('h')

In [ ]:
df